In [ ]:
from NPSmethods2 import*
import numpy as np
import glob
import pickle as pkl
import tensorflow as tf

In [ ]:
#Function that applies fourier transforms to OD images to get a noise power spectrum
def calcNoisePowSpecs(ODs):

    noisePowSpecs = []
    for k in range(len(ODs)):
        noise = ODs[k]
        noiseFFT = np.fft.fftshift(np.fft.fft2(np.fft.fftshift(noise)))
        noisePowSpec = np.abs(noiseFFT)**2
        noisePowSpec = noisePowSpec / ODs[k].sum()
        noisePowSpec[noisePowSpec.shape[0]//2, noisePowSpec.shape[1]//2] = 0
        _, _, K_x, K_y = getFreq(imgSysData["CCDPixelSize"], imgSysData["magnification"], [100, 100])
        d = imgSysData["wavelen"] / (2*np.pi*imgSysData["NA"]) 
        M2k_Fit_fake = M2kFuncAnal(K_x, K_y, d, 1.5, 1, .5, -1.6, 0, -1)
        bkg = np.mean(noisePowSpec[M2k_Fit_fake==0])
        noisePowSpec -= bkg
        noisePowSpec[noisePowSpec < 0] = 0
        noisePowSpec[M2k_Fit_fake == 0] = 0
    
        noisePowSpecs.append(noisePowSpec)

    return noisePowSpecs

In [ ]:
imgSysData = { 
    "CCDPixelSize": 13,      # pixel size of the CCD, in micron 
    "magnification": 27,      # 799.943 / 29.9099, # magnification of the imaging system 
    "wavelen"     : 0.852,     # wavelength of the imaging beam, in micron 
    "NA"          : 0.37,      # numerical aperture of the objective 
    "ODtoAtom"    : 13
}

choices = {
    "if_Save"                     : False,
    "do_Fit"                      : True,
    "plot_2dGas"                  : False,
    "plot_NoisePowSpec"           : False,
    "plot_NoisePowSpec_LineCut"   : False,
    "plot_Pupil"                  : False,
    "plot_PSF"                    : False,
    "plot_PSF_LineCut"            : False,
    "plot_PSF_abs2"               : False,
    "plot_PSF_abs2_LineCut"       : False,
    "plot_Sk"                     : False,
    "normalize"                   : False
}

path = r'C:\Users\eric0\Jupyter Notebooks\Cold Physics\raw_image' #Path to raw images
savepath = r'C:\Users\eric0\Jupyter Notebooks\Cold Physics\results' #Path to save processes images and parameters
numOfImgsInEachRun = 1 #Number of images
rampingParameter = 0
trapRegion = (slice(100, 200), slice(100, 200))
noiseRegion = (slice(0, 300), slice(0, 300))

In [ ]:
parameters = []
images = []

#Sort through folders and get parameters and images for each folder
for folder in glob.iglob(path + "\\*"):
    atomODs, atomODAvg, _, _, _, _ = \
        readInImages(folder + "\\", numOfImgsInEachRun, rampingParameter, trapRegion, noiseRegion)
    noisePowSpecs = calcNoisePowSpecs(atomODs)
    _, _, _, _, popt, _, _ = doCalibration(folder, savepath, trapRegion, noiseRegion, numOfImgsInEachRun, rampingParameter, imgSysData, choices)
    for noisePowSpec in noisePowSpecs:
        images.append(noisePowSpec)
    for i in range(len(noisePowSpecs)):
        parameters.append(popt)

In [ ]:
#Cut the images down to a smaller 49x49 pixel size
images = np.delete(images, np.s_[74:100], axis = 1)
images = np.delete(images, np.s_[0:25], axis = 1)
images = np.delete(images, np.s_[74:100], axis = 2)
images = np.delete(images, np.s_[0:25], axis = 2)

In [ ]:
#Check image dimensions
print(np.shape(images))

In [ ]:
#Save images to a folder
for imnum in range(len(images)):
    K_x, K_y, _, _ = getFreq(imgSysData["CCDPixelSize"], imgSysData["magnification"], (50,50))
    fig = plt.figure(figsize=(5.5, 4.5))
    ax = fig.add_subplot(111)
    im = ax.pcolor(K_x, K_y, images[imnum], cmap=cm.jet, vmin = 0, vmax = 0.5)
    plt.colorbar(im)
    fig.savefig("C:\\Users\\eric0\\OneDrive\\Desktop\\Cold Physics\\real_NPS_ims\\real_NPS_%d" % imnum)
    plt.close(fig)

In [ ]:
#Convert images and parameters to arrays and delete any samples that have negative pixel values (should be 0)
images = np.array(images).reshape((len(images),-1))
parameters = np.array(parameters)
print(np.argwhere(np.amin(images, axis=1) < 0))
parameters = np.delete(parameters, np.where(np.amin(images, axis=1) < 0), 0)
images = np.delete(images, np.where(np.amin(images, axis=1) < 0), 0)

In [ ]:
for image in images[0:20]:
    plt.hist(image.flatten(), density=1, range=(0,.25))
    plt.show()

In [ ]:
#Separate out the defocus parameter and sets of 3 consecutive and 5 consecutive images and their parameters
defocus = parameters[:,5]
threeimages = []
fiveimages = []
threeimparams = []
fiveimparams = []
for i in range(len(images)):
    if (i + 1) % 3 == 0:
        if defocus[i] == defocus[i - 2]:
            threeimages.append(sum(images[i - 2:i] / 3))
            threeimparams.append(defocus[i])
                               
    if (i + 1) % 5 == 0:
        if defocus[i] == defocus[i - 4]:
            fiveimages.append(sum(images[i - 4:i] / 5))
            fiveimparams.append(defocus[i])
threeimages = np.array(threeimages)
fiveimages = np.array(fiveimages)

In [ ]:
#Normalize image sets
# images /= 20
# threeimages /= 20
# fiveimages /= 20
# mean = pkl.load(open(savepath + "\\mean", "rb"))
# std = pkl.load(open(savepath + "\\std", "rb"))
# images[images < 0.001] = 0.001
# threeimages[threeimages < 0.001] = 0.001
# fiveimages[fiveimages < 0.001] = 0.001
# images = np.log10(images)
# threeimages = np.log10(threeimages)
# fiveimages = np.log10(fiveimages)
# images /= 3
# threeimages /= 3
# fiveimages /= 3
for i in range(len(images)):
    if np.std(images[i])==0:
        print(i)
        continue
    images[i] -= np.mean(images[i])
    images[i] /= np.std(images[i])
for i in range(len(threeimages)):
    threeimages[i] -= np.mean(threeimages[i])
    threeimages[i] /= np.std(threeimages[i])
for i in range(len(fiveimages)):
    fiveimages[i] -= np.mean(fiveimages[i])
    fiveimages[i] /= np.std(fiveimages[i])

In [ ]:
for image in images[0:20]:
    plt.hist(image.flatten(), density=1)
    plt.show()

In [ ]:
#Scale parameters using scaler used in artificial data normalization for ML validation set
yscaler = pkl.load(open(savepath + "\\yscaler", "rb"))
defocus = defocus.reshape(-1, 1)
parameters_s = yscaler.transform(defocus)

In [ ]:
#Convert images and parameters to tensorflor tensors
images = tf.convert_to_tensor(images, dtype=tf.float32)
threeimages = tf.convert_to_tensor(threeimages, dtype=tf.float32)
fiveimages = tf.convert_to_tensor(fiveimages, dtype=tf.float32)
parameters = tf.convert_to_tensor(parameters, dtype=tf.float32)
threeimparams = tf.convert_to_tensor(threeimparams, dtype=tf.float32)
fiveimparams = tf.convert_to_tensor(fiveimparams, dtype=tf.float32)

In [ ]:
#Store images and parameters as pickle files
pkl.dump(images, open(savepath + "\\realdatain", "wb"))
pkl.dump(threeimages, open(savepath + "\\3images", "wb"))
pkl.dump(fiveimages, open(savepath + "\\5images", "wb"))
pkl.dump(parameters, open(savepath + "\\realdataout", "wb"))
pkl.dump(threeimparams, open(savepath + "\\3imparams", "wb"))
pkl.dump(fiveimparams, open(savepath + "\\5imparams", "wb"))
pkl.dump(parameters_s, open(savepath + "\\realdataout_s", "wb"))